In [15]:
import numpy as np
import os
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.preprocessing.sequence import pad_sequences

In [16]:
dirs = os.listdir('./aclImdb/train')

In [17]:
text = []
labels = []
for i in dirs:
    if i == 'neg':
        for j in os.listdir('./aclImdb/train/neg'):
            with open('./aclImdb/train/neg/'+j, 'rb') as f:
                text.append(f.read())
                labels.append([1,0])
    if i == 'pos':
        for j in os.listdir('./aclImdb/train/pos'):
            with open('./aclImdb/train/pos/'+j, 'rb') as f:
                text.append(f.read())
                labels.append([0,1])

In [18]:
my_set = set()
for x in text:
    for i in str(x).split(' '):
        my_set.add(i)

In [19]:
word_dict = {}
for index, word in enumerate(my_set):
    word_dict[word] = index

In [20]:
def encode(text):
    vector = []
    for word in text.split(' '):
        vector.append(word_dict[word])
    return vector

In [21]:
#vector takes fixed lenght i/p so we pad remaining
reviews = []
for review in text[:1000]:
    reviews.append(encode(str(review)))

In [22]:
reviews = pad_sequences(reviews,maxlen=100,value=0.)

In [29]:
model = Sequential()
model.add(Embedding(len(my_set)+1, 128, input_length=100))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

In [30]:
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(reviews, np.array(labels[:1000]),
          batch_size=128,
          epochs=4)

Epoch 1/4
1000/1000 [==============================] - 21s - loss: 0.6543 - acc: 0.9270    
Epoch 2/4
1000/1000 [==============================] - 11s - loss: 0.3848 - acc: 1.0000    
Epoch 3/4
1000/1000 [==============================] - 11s - loss: 0.0107 - acc: 1.0000    
Epoch 4/4
1000/1000 [==============================] - 10s - loss: 0.0014 - acc: 1.0000    


In [32]:
model.save('bilstm.h5')